In [ ]:
###

### 値の種類が1種類以下のものを除外
drop_cols = train_df.nunique()[train_df.nunique() <= 1].index
train_df = train_df.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)


def preprocess(df, flag):
    
    # 取引時点
    replaced = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99",
    }
    d = {}
    for val in df["取引時点"].value_counts().keys():
        new_float = 0.0
        for k, v in replaced.items():
            if k in val:
                new_float = float(val.replace(k, v))
        d[val] = new_float
    df["取引時点"] = df["取引時点"].replace(d)

    # 用途
    col = "用途"
    d = {}
    for val in df[col].value_counts().keys():
        if "、" in val:
            d[val] = val.split("、")[0]
    df[col] = df[col].replace(d)

    # 建築年
    col = "建築年"
    d = {}
    for val in df[col].value_counts().keys():
        if "平成" in val:
            _year = int(val.split("平成")[1].split("年")[0])
            year = 1988 + _year
        if "昭和" in val:
            _year = int(val.split("昭和")[1].split("年")[0])
            year = 1925 + _year
        if "令和" in val:
            _year = int(val.split("令和")[1].split("年")[0])
            year = 2018 + _year
        d[val] = year
    d["戦前"] = 1945
    df[col] = df[col].replace(d)
    
    # col = "間取り"
    col = "間取り"
    ints = ["１", "２", "３", "４", "５", "６", "７", "８", "９"]

    d1 = {}
    dr = defaultdict(lambda: defaultdict(int))
    for k in sorted(df[col].value_counts().keys()):
        if k[0] in ints:
            d1[k] = int(k[0])
        else:
            d1[k] = 0

        for room_type in ["Ｌ", "Ｄ", "Ｋ", "Ｓ"]:
            dr[room_type][k] = k.count(room_type)

    df[f"{col}_otherroom"] = df[col].replace(d1)
    df[f"{col}_L"] = df[col].replace(dr["Ｌ"])
    df[f"{col}_D"] = df[col].replace(dr["Ｄ"])
    df[f"{col}_K"] = df[col].replace(dr["Ｋ"])
    df[f"{col}_S"] = df[col].replace(dr["Ｓ"])
    df.drop(col, axis=1, inplace=True)

    # 面積
    col = "面積（㎡）"
    if flag == "train":
        df[col] = df[col].replace({"2000㎡以上": 2000})
    df[col] = df[col].astype(int)

    # 距離
    col = "最寄駅：距離（分）"
    d = {
        "30分?60分": 45,
        "1H?1H30": 75,
        "2H?": 120,
        "1H30?2H": 105,
    }
    df[col] = df[col].replace(d)
    df[col] = df[col].astype(float)

    for col in ["用途", "地区名", "最寄駅：名称", "建物の構造", "今後の利用目的", "都市計画"]:
        df[col] = df[col].astype("category")
    
    return df